In [1]:
!pip install datasets evaluate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00


In [ ]:
# !pip install accelerate

In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:00


In [3]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.4 MB/s eta 0:00:00


In [4]:
import transformers

print(transformers.__version__)

# # Fine-tuning a model on a text classification task

task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 1

# ## Loading the dataset

# We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.

from datasets import load_dataset, load_metric
import evaluate
# Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

# actual_task = "mnli" if task == "mnli-mm" else task
# dataset = load_dataset("glue", actual_task)

import pickle

test_file = 'llama_reddit_100_test.pickle'
# test_file = 'llama_reddit_100_test.pickle'
# test_file = 'llama_solid_extended_100_test.pickle'
# test_file = 'llama_default_reddit_100_test.pickle'
# test_file = 'llama_default_gab_100_test.pickle'
# test_file = 'llama_default_solid_extended_100_test.pickle'

# test_files = ['llama_reddit_100_test.pickle', 'llama_reddit_100_test.pickle']
with open(test_file, 'rb') as handle:
    test = pickle.load(handle)


def mapping_function(example):
    if example['labels'] == "HATEFUL":
        example['labels'] = 1
    else:
        example['labels'] = 0
    return example


import pandas as pd
data = pd.DataFrame({"input": test['predictions'], "labels": test['real_answers']})
from datasets import Dataset
dataset = Dataset.from_pandas(data)
dataset = dataset.map(mapping_function)
trainval_test_dataset = dataset.train_test_split(test_size=int(0.2*len(dataset)), seed=42)


test_dataset = trainval_test_dataset['test']
trainval_dataset = trainval_test_dataset['train']

train_val_dataset = trainval_dataset.train_test_split(test_size=int(0.2*len(trainval_dataset)), seed=42)

dataset = train_val_dataset
# dataset = load_dataset("llama_bfsc_lm_output_data")
# metric = load_metric('glue', actual_task)
metric = evaluate.load("f1")

4.30.2


Map:   0%|          | 0/95 [00:00<?, ? examples/s]

In [5]:
dataset['train'][0]

{'input': '<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk>NEUT',
 'labels': 0}

In [6]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

# You can directly call this tokenizer on one sentence or a pair of sentences:

tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

def preprocess_function(example):
    return tokenizer(example['input'], truncation=True)

preprocess_function(dataset['train'][:5])


encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset['train']['input']

In [17]:

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2

num_labels = 2

# device_no = 0
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


# metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
metric_name = "f1"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_steps=10
    # push_to_hub=True,
)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # if task != "stsb":
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Then we just need to pass all of this along with our datasets to the `Trainer`:

# validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"

validation_key = "test"

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.we

In [18]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 61
    })
    test: Dataset({
        features: ['input', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 15
    })
})

In [19]:
# !pip install accelerate==0.20.3


In [20]:
trainer.train()

# We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,F1
10,0.691700,0.612530,0.000000
20,0.639800,0.449160,0.000000
30,0.511500,0.237708,0.000000
40,0.245100,0.139483,0.000000
50,0.195700,0.119342,0.000000
60,0.358900,0.085519,0.000000
70,0.147300,0.070493,0.000000
80,1.036700,0.063181,0.000000
90,0.428000,0.048658,1.000000
100,0.681400,0.037380,1.000000


TrainOutput(global_step=183, training_loss=0.43879843792303014, metrics={'train_runtime': 7.6356, 'train_samples_per_second': 23.967, 'train_steps_per_second': 23.967, 'total_flos': 1812371342940.0, 'train_loss': 0.43879843792303014, 'epoch': 3.0})

In [21]:
model.save_pretrained(f"{model_name}-finetuned-{task}")

In [22]:
!ls -la distilbert-base-uncased-finetuned-cola/pytorch_model.bin

-rw-r--r-- 1 root root 267855533 Jul 10 21:33 distilbert-base-uncased-finetuned-cola/pytorch_model.bin


In [13]:
trainer.train()

# We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,0.104561,0.000000
2,No log,0.008261,1.000000
3,No log,0.004983,1.000000
4,No log,0.002677,1.000000
5,No log,0.005337,1.000000
6,No log,0.002924,1.000000
7,No log,0.002802,1.000000
8,No log,0.002270,1.000000
9,0.402100,0.002276,1.000000
10,0.402100,0.002195,1.000000


TrainOutput(global_step=1525, training_loss=0.3951768870822719, metrics={'train_runtime': 93.8885, 'train_samples_per_second': 16.243, 'train_steps_per_second': 16.243, 'total_flos': 15103094524500.0, 'train_loss': 0.3951768870822719, 'epoch': 25.0})

In [22]:
trainer.evaluate()

{'eval_loss': 0.43620380759239197,
 'eval_runtime': 0.1362,
 'eval_samples_per_second': 117.516,
 'eval_steps_per_second': 117.516,
 'epoch': 5.0}

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
!cp distilbert-base-uncased-finetuned-cola/config.json /content/drive/MyDrive/ml-models/llama_training_reddit_colab_1_ep_128b/

In [28]:
!cp -r distilbert-base-uncased-finetuned-cola/runs /content/drive/MyDrive/ml-models/llama_training_reddit_colab_1_ep_128b/bfsc_model/

In [11]:
# To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

# You can now upload the result of the training to the Hub, just execute this instruction:

trainer.push_to_hub()

# You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:
#
# ```python
# from transformers import AutoModelForSequenceClassification
#
# model = AutoModelForSequenceClassification.from_pretrained("sgugger/my-awesome-model")
# ```

# ## Hyperparameter search

# The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it.

# ! pip install optuna
# ! pip install ray[tune]

# During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We jsut use the same function as before:

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

# And we can instantiate our `Trainer` like before:

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# The method we call this time is `hyperparameter_search`. Note that it can take a long time to run on the full dataset for some of the tasks. You can try to find some good hyperparameter on a portion of the training dataset by replacing the `train_dataset` line above by:
# ```python
# train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10)
# ```
# for 1/10th of the dataset. Then you can run a full training on the best hyperparameters picked by the search.

best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

# The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

best_run

# You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.
#
# To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()






╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3716 in push_to_hub              │
│                                                                                                  │
│   3713 │   │   # If a user calls manually `push_to_hub` with `self.args.push_to_hub = False`, w  │
│   3714 │   │   # it might fail.                                                                  │
│   3715 │   │   if not hasattr(self, "repo"):                                                     │
│ ❱ 3716 │   │   │   self.init_git_repo()                                                          │
│   3717 │   │                                                                                     │
│   3718 │   │   model_name = kwargs.pop("model_name", None)                                       │
│   3719 │   │   if model_name is None and self.args.should_save:                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3566 in init_git_repo            │
│                                                                                                  │
│   3563 │   │   else:                                                                             │
│   3564 │   │   │   repo_name = self.args.hub_model_id                                            │
│   3565 │   │   if "/" not in repo_name:                                                          │
│ ❱ 3566 │   │   │   repo_name = get_full_repo_name(repo_name, token=self.args.hub_token)          │
│   3567 │   │                                                                                     │
│   3568 │   │   # Make sure the repo exists.                                                      │
│   3569 │   │   create_repo(repo_name, token=self.args.hub_token, private=self.args.hub_private_  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:828 in get_full_repo_name      │
│                                                                                                  │
│    825                                                                                           │
│    826 def get_full_repo_name(model_id: str, organization: Optional[str] = None, token: Optiona  │
│    827 │   if organization is None:                                                              │
│ ❱  828 │   │   username = whoami(token)["name"]                                                  │
│    829 │   │   return f"{username}/{model_id}"                                                   │
│    830 │   else:                                                                                 │
│    831 │   │   return f"{organization}/{model_id}"                                               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:118 in _inner_fn    │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │                                                                                      │
│ ❱ 118 │   │   return fn(*args, **kwargs)                                                         │
│   119 │                                                    